<a href="https://colab.research.google.com/github/karaage0703/colab-notebooks/blob/main/Tanuki_data_synthesis_from_elyza_tasks_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データ合成

## 参考
- https://note.com/npaka/n/ndbbd243f8073
- https://github.com/axolotl-ai-cloud/axolotl/issues/911
- https://colab.research.google.com/drive/114CRa-xh0yh-jVzzSav2VL3vqBOHSgr9?hl=ja#scrollTo=Rqy-n7Fi9mDb

セットアップ

Google Colabのシークレットキーに`HF_TOKEN`という名前でHugging Faceのトークンを追加しておいてください。

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

# シークレットキーを使わない場合は以下に直接入力してください
# HF_TOKEN = "xxx"

ライブラリセットアップ

In [ ]:
!pip install -qqq -U transformers accelerate bitsandbytes
!pip install -qqq flash_attn --no-build-isolation
#!pip install flash-attn==2.3.3
!pip install -qqq autoawq

In [ ]:
!pip install pyarrow==14.0.1
!pip install torch==2.4.0 torchvision==0.19.0+cu121 torchaudio==2.4.0+cu121

Tanuki-8x8Bのセットアップ

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name = "team-hatakeyama-phase2/Tanuki-8x8B-dpo-v1.0-AWQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

Tanuki-8x8Bのテスト

In [ ]:
# メッセージの準備
messages = [
    {"role": "system", "content": "あなたはAIアシスタントです。"},
    {"role": "user", "content": "こんにちは"}
]

# 推論の実行
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)
output_ids = model.generate(
    input_ids,
    max_new_tokens=512,
    temperature=0.5,
    #streamer=streamer
)

output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

## データセットの変換

ELYZA-tasks-100のロード

In [ ]:
from datasets import load_dataset

datasets = load_dataset("elyza/ELYZA-tasks-100")

In [ ]:
datasets

システムプロンプト設定

In [ ]:
system_prompt = r"""
あなたには試験の練習問題をつくってもらいたいです。

プロンプトには input, outputが含まれていて、 inputには質問、outputには答えが入っています。
これと似たような問題が出題されるので、練習問題として input, outputを参考に、新しい質問(input)と答え(output)のセットを1つ作成してください。
inputとoutputは、元の問題とは違う、新しいものにしてください。

出力形式は以下としてください。

input: ###
{input}
###

output: ###
{output}
""".strip()

データセットからプロンプトを生成

In [ ]:
from tqdm import tqdm
prompt_list = []

for dt in tqdm(datasets["test"]):
    prompt = f"input: ###\n{dt['input']}\n###\n\noutput: ###\n{dt['output']}\n###"
    prompt_list.append(prompt)

データ確認

In [ ]:
prompt_list[0]

In [ ]:
#test_prompt_list = prompt_list[:2]

In [ ]:
from tqdm import tqdm

results = []

for i, prompt in enumerate(tqdm(prompt_list), start=1):
#for i, prompt in enumerate(tqdm(test_prompt_list), start=1):
    print(f"number: {i}")

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    # 推論の実行
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=1024,
        temperature=1.0,
    )

    #text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    text = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1) :], skip_special_tokens=True)
    input_text = ""
    output_text = ""

    try:
        input_text = text.split("input: ###")[1].split("###")[0].strip()
        output_text = text.split("output: ###")[1].split("###")[0].strip()
    except Exception as e:
        print(e)

    # 結果を辞書形式で保存
    results.append({"id": i, "text": text, "input": input_text, "output": output_text})

In [ ]:
results[0]

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_excel("elyza_tasks_100_new.xlsx", index=False)